#Bepalen van de waarde van de parameters EPS en MinSamples voor de DBScan

In dit script creëren we een nieuwe methode om te bepalen wat de waarde moet zijn van de parameters EPS en MinSamples. Deze parameters worden door de DBScan gebruikt om clusters te maken. EPS is de afkorting voor epsilon en staat voor de maximale afstand tussen twee XY punten om tot hetzelfde cluster te horen. MinSamples staat voor het minimale aantal XY punten die bij elkaar in buurt moeten liggen om tot een cluster beschouwd te worden.  

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import preprocessing

In [ ]:
#inladen csv (resultaat script "00_bereken_afstand_snelheid_tussen_XYT_punten.ipynb")
df = pd.read_csv('XYT_speed_distance.csv')
df = df[:-1] #Laatste regel is NAN
#df_org = df #Orginele df ivm merge

df.head(5)

In [ ]:
#Schalen van XYV punten 
coords_speed = np.asarray(df[['Lat_a', 'Lon_a', "distance_km", "VgNr"]])
coords = preprocessing.scale(coords_speed)

df.head(10)

In [ ]:
def gemiddelde_kmu_per_eps(df, coords, eps, min_samples, speed_column="speed_kmu"):
    model = DBSCAN(eps=eps, min_samples=min_samples, algorithm="auto").fit(coords)
    cluster_labels = model.labels_
    num_clusters = len(set(cluster_labels))
    df["cluster"] = cluster_labels
    clusters = df.speed_kmu[df["cluster"] > -1]   
    ruis =df.speed_kmu[df["cluster"] == -1]
    u, prob = mannwhitneyu(clusters, ruis)
    kmu_per_cluster = df.groupby('cluster')["speed_kmu"].mean().reset_index(name='kmu_per_cluster')
    mean_kmu = kmu_per_cluster["kmu_per_cluster"].mean()
    return (eps, min_samples, num_clusters, mean_kmu, u)
    

#     kmu_per_cluster = df.groupby('cluster')["speed_kmu"].mean()
#     .reset_index(name='kmu_per_cluster')
#     mean_kmu = kmu_per_cluster["kmu_per_cluster"].mean()
#     return (eps, min_samples, num_clusters, kmu_per_cluster, u)
# y = gemiddelde snelheid per cluster
# x = eps
# DataFrame.append

# output_df = pd.DataFrame()
output_list = []

for epsilon in np.linspace(0.14, 0.5, 10):
    for min_samples in range(1, 8, 1):
        output = gemiddelde_kmu_per_eps(df=df, coords=coords, eps=epsilon, min_samples=min_samples)
        output_list.append(output)

# for epsilon in range(1, 1000, 1):
#     for min_samples in range(1, 6, 1):
#         output = gemiddelde_kmu_per_eps(df=df_loop, coords=coords_2, eps=epsilon/100000, min_samples=min_samples)
#         output_list.append(output)

output_df = pd.DataFrame(output_list, columns = ["eps", "min_samples", "NoP_cluster", "mean_kmu_speed", "MannWhitney_U"])

# gemiddelde_kmu_per_eps(mean_kmu)

In [ ]:
output_df

In [ ]:
x = np.array(output_df["eps"])
y = np.array(output_df["min_samples"])
z = np.array(output_df["MannWhitney_U"])
hoi = pd.DataFrame.from_dict(np.array([x,y,z]).T)
hoi.columns = ['x','y','z']
pivotted = hoi.pivot('y','x','z')

import seaborn as sns
sns.set()

sns.heatmap(pivotted, cmap="RdBu")

In [ ]:
hoi.sort_values(by="z")
hoi.to_csv("VGNR_hoi.csv")

In [ ]:
# # from bokeh.plotting import figure, show, output_file
# from bokeh.plotting import figure, output_file, show
# from bokeh.io import push_notebook, show, output_notebook, curdoc, show
# from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid
# from bokeh.models.glyphs import VBar
# output_notebook()

# #KMU Visual
# x = output_df["eps"]
# y = output_df["mean_kmu_speed"]

# # output to static HTML file
# output_file("line.html")

# p = figure(plot_width=500, plot_height=500)
# p.circle(x=x, y=y, size=, color="navy")
# show(p)

# #NoP visual
# x = output_df["eps"]
# y = output_df["NoP_cluster"]

# # output to static HTML file
# output_file("line.html")

# p = figure(plot_width=500, plot_height=500)
# p.circle(x=x, y=y, size=20, color="navy")
# show(p)